# Introduction

This tutorial focuses on implementation of alternative to sigmoid transfer functions (sometimes also called activation functions or non-linearities). First, we will implement another type of sigmoidal-like activation - hyperboilc tangent (Tanh) (to date, we have implemented sigmoid (logistic) activation). Then we go to unbounded (or partially bounded) activations: Rectifying Linear Units (ReLU) and Maxout.

## Virtual environments

Before you proceed onwards, remember to activate your virtual environment by typing `activate_mlp` or `source ~/mlpractical/venv/bin/activate` (or if you did the original install the "comfy way" type: `workon mlpractical`).


## Syncing the git repository

Look <a href="https://github.com/CSTR-Edinburgh/mlpractical/blob/master/gitFAQ.md">here</a> for more details. But in short, we recommend to create a separate branch for this lab, as follows:

1. Enter the mlpractical directory `cd ~/mlpractical/repo-mlp`
2. List the branches and check which are currently active by typing: `git branch`
3. If you have followed our recommendations, you should be in the `lab4` branch, please commit your local changed to the repo index by typing:
```
git commit -am "finished lab4"
```
4. Now you can switch to `master` branch by typing: 
```
git checkout master
 ```
5. To update the repository (note, assuming master does not have any conflicts), if there are some, have a look <a href="https://github.com/CSTR-Edinburgh/mlpractical/blob/master/gitFAQ.md">here</a>
```
git pull
```
6. And now, create the new branch & swith to it by typing:
```
git checkout -b lab5
```

# Overview of alternative transfer functions

Now, we briefly summarise some other possible choices for hidden layer transfer functions.

## Tanh

(1) $h_i(a_i) = \mbox{tanh}(a_i) = \frac{\exp(a_i) - \exp(-a_i)}{\exp(a_i) + \exp(-a_i)}$

Hence, the derivative h_i w.r.t a_i is:

(2) $
\frac{\partial h_i}{\partial a_i} = 1 - h^2_i
$


## ReLU

Given linear activation $a_{i}$ relu implements the following operation:

(3) $h_i(a_i) = \max(0, a_i)$

Hence, the gradient is :

(4) $\begin{align}
\frac{\partial h_i}{\partial a_i} &=
\begin{cases}
     1     & \quad \text{if } a_i \geq 0 \\
     0       & \quad \text{if } a_i < 0 \\
\end{cases}
\end{align}
$

ReLU implements a form of data-driven sparsity, that is, on average the activations are sparse (many of them are 0) but the general sparsity pattern will depend on particular data-point. This is different from sparsity obtained in model's parameters one can obtain with L1 regularisation as the latter affect all data-points in the same way.

## Maxout

Maxout is an example of data-driven type of non-linearity in which "the transfer function" can be learn from data. That is, the model can build non-linear transfer-function from piece-wise linear components. Those (linear components), depending on the number of linear regions used in the pooling operator ($K$ parameter), can approximate an arbitrary functions, like ReLU, abs, etc.

The maxout non-linearity, given some sub-set (group, pool) of $K$ linear activations $a_{j}, a_{j+1}, \ldots, a_{j+K}$ at the layer $l$-th, implements the following operation:

(5) $h_i(a_j, a_{j+1}, \ldots, a_{j+K}) = \max(a_j, a_{j+1}, \ldots, a_{j+K})$

Hence, the gradient of $h_i$ w.r.t to the pooling region $a_{j}, a_{j+1}, \ldots, a_{j+K}$  is :

(6) $\begin{align}
\frac{\partial h_i}{\partial (a_j, a_{j+1}, \ldots, a_{j+K})} &=
\begin{cases}
     1     & \quad \text{for maxed activation}  \\
     0       & \quad \text{otherwsie} \\
\end{cases}
\end{align}
$

Implemenation tips are given in the Exercise 3.

# On the weight initialisation

Activation functions directly affect the "network's dynamic", that is, the magnitudes of the statistics each layer is producing. For example, *slashing* non-linearities like sigmoid or tanh bring the linear activations to a certain bounded range. ReLU, contrary has unbounded positive side. This directly affects all statistics collected in forward and backward passes as well as the gradients w.r.t paramters hence also the pace at which the model learns. That is why learning rate usually requires to be tuned for certain characterictics of the non-linearities. 

The other, to date mostly "omitted" in the lab hyper-parameter, is the initial range the weight matrices are initialised with. For sigmoidal non-linearities (sigmoid, tanh) it is an important hyper-parameter and considerable amount of research has been put into determining what is the best strategy for choosing it. In fact, one of the early triggers of the recent resurgence of deep learning was pre-training - techniques allowing to better initialise the weights in unsupervised manner so one can effectively train deeper models in supervised fashion later.  

## Sigmoidal transfer functions

Y. LeCun in [Efficient Backprop](http://link.springer.com/chapter/10.1007%2F3-540-49430-8_2) paper for sigmoidal units recommends the following setting (assuming the data has been normalised to zero mean, unit variance): 

(7) $ r = \frac{1}{\sqrt{N_{IN}}} $

where $N_{IN}$ is the number of inputs to the given layer and the weights are then sampled from (usually uniform) distribution $U(-r,r)$. The motivation is to keep the initial forward-pass signal in the linear region of the sigmoid non-linearity so the gradients are large enough for training to proceed (notice the sigmoidal non-linearities sature when activations get either very positive or very negatvie leading to very small gradients and hence poor learning dynamics as a result).

Initialisation used in (7) however leads to different magnitues of activations/gradients at different layers (due to multiplicative narute of performed comutations) and more recently, [Glorot et. al](http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf) proposed so called *normalised initialisation*, which ensures the variance of the forward signal (activations) is approximately the same in each layer. The same applies to the gradients obtained in backward pass.  

The $r$ in the *normalised initialisation* for $\mbox{tanh}$ non-linearity is then:

(8) $ r = \frac{\sqrt{6}}{\sqrt{N_{IN}+N_{OUT}}} $

For sigmoid (logistic) non-linearity, to get similiar characteristics, one should scale $r$ in (8) by 4, that is:

(9) $ r = \frac{4\sqrt{6}}{\sqrt{N_{IN}+N_{OUT}}} $

## Piece-wise linear transfer functions (ReLU, Maxout)

For unbounded transfer functions initialisation is not that crucial as with sigmoidal ones. It's due to the fact their gradients do not diminish (they are acutally more likely to explode) and they do not saturate (ReLU saturates at 0, but not on the positive slope, where gradient is 1 everywhere).


# Exercise 1: Implement Tanh

Implementation should follow the conventions used to build other layer types (for example, Sigmoid and Softmax). Test the solution by training one-hidden-layer (100 hidden units) model (similiar to the one used in Task 3a in the coursework). 

Tune the learning rate and compare the initial ranges in equations (7) and (8) (notice, there might not be much difference for one-hidden-layer model, but you can easily notice substantial gain from using (8) (or (9) for logistic activation) for deeper models, for example, 5 hidden-layer one from the first coursework).

Implementation tip: Use numpy.tanh() to compute non-linearity.  Use irange argument when creating the given layer type to provide the initial sampling range.

# Exercise 2: Implement ReLU

Implementation should follow the conventions used to build Linear, Sigmoid and Softmax layers. Test the solution by training one-hidden-layer (100 hidden units) model (similiar to the one used in Task 3a in the coursework). Tune the learning rate (start with the initial one set to 0.1) and the initial weight range set to 0.05.

# Exercise 3: Implement Maxout

Implementation should follow the conventions used to build Linear, Sigmoid and Softmax layers. Implement scenario with non-overlapping pooling regions. Test the solution by training a one-hidden-layer model with the total number of weights similar to the models used in the previous exercises. Use the same optimisation hyper-parameters (learning rate, initial weights range) as you used for ReLU models. Tune the pool size $K$ (but keep the number of total parameters fixed).

Note: Max operator reduces dimensionality, hence for example, to get 100 hidden maxout units with pooling size set to $K=2$ the size of linear part needs to be set to 100*K (assuming non-overlapping pools). This affects how you compute the total number of weights in the model.

Implementation tips: To back-propagate through maxout layer, one needs to keep track of which linear activation $a_{j}, a_{j+1}, \ldots, a_{j+K}$ was maxed-out in each pool. The convenient way to do so is by storing the maxed indices in fprop function and then in back-prop stage pass the gradient only through those (i.e. for example, one can build an auxiliary matrix where each element is either 1 (if unit was passed-forward through max operator for a given data-point) or 0 otherwise. Then in backward pass it suffices to upsample the maxout *igrads* signal to linear layer dimension and element-wise multiply by the aforemenioned auxiliary matrix.

# Exercise 4: Train all the above models with dropout

Try all of the above non-linearities with dropout training. Use the dropout hyper-parameters $\{p_{inp}, p_{hid}\}$ that worked best for sigmoid models from the previous lab.

Note: the code for dropout you were asked to implement last week has not been given as a solution for this week - as a result you need to move/merge the required dropout parts from your previous *lab4* branch (or implement it if you haven't done it so far). 
